# Script Machine Debugging Tool (Libbitcoin c++)

<br>
<img src="images/script_evaluation_bip_16_141.jpg" alt="drawing" style="" width="800px"/>


<hr style="border: 0.5px dashed #000;">

In [1]:
// Compiler & linker information for c++ interpreter.
#pragma cling add_include_path("/usr/local/include")
#pragma cling add_library_path("/usr/local/lib")
#pragma cling load("bitcoin","secp256k1","pthread","boost_chrono-mt","boost_date_time-mt","boost_filesystem","boost_iostreams-mt","boost_locale-mt","boost_log-mt","boost_program_options-mt","boost_regex-mt","boost_system","boost_thread-mt")

// Libbitcoin-System.
#include <bitcoin/bitcoin.hpp> 

// Libbitcoin-Script Machine Script.
#include "supporting_scripts/script_machine.cpp"

### 1. Previous transaction output (UTXO).

**Previous output script** (fetch by `txid` & `index`)

In [2]:
!bx fetch-tx 8231a9027eca6f2bd7bdf712cd2368f0b6e8dd6005b6b348078938042178ffed -f json | jq -r  ".transaction.outputs[0].script" | bx script-encode


a914dcdc2f89b96c420751e3750da7d5073a81b1694687


In [3]:
auto previous_output_script_data = bc::to_chunk(
  bc::base16_literal("a914dcdc2f89b96c420751e3750da7d5073a81b1694687"));
bc::chain::script previous_output_script;
previous_output_script.from_data(previous_output_script_data, false);


**Previous output amount** (fetch by `txid` & `index`)

In [4]:
!bx fetch-tx 8231a9027eca6f2bd7bdf712cd2368f0b6e8dd6005b6b348078938042178ffed -f json | jq -r  ".transaction.outputs[0].value"


129800000


In [5]:
uint64_t previous_output_amount = 129800000;


### 2. Spending transaction.

In [6]:
// Paste your raw transaction data.
auto spending_transaction_data = bc::to_chunk(
  bc::base16_literal("01000000000101edff78210438890748b3b60560dde8b6f06823cd12f7bdd72b6fca7e02a9318200000000171600149a19a31c2fda7d0c30215ec954a20a542aa84ad3ffffffff016003b807000000001976a914bbef244bcad13cffb68b5cef3017c7423675552288ac0247304402207ecbb796a2bc706d90e2ed7efb58f59822bdc4c253b91f6eecd26ca5df1a6bb60220700b737f3c49b2f21bb228fadeab786e2ac78fd87890ede3f5d299e81880d9630121026ccfb8061f235cc110697c0bfb3afb99d82c886672f6b9b5393b25a434c0cbf300000000"));
bc::chain::transaction spending_transaction;
auto tx_set = spending_transaction.from_data(spending_transaction_data, true, true); // wire=true, witness=true


**Index of spending input.**

In [7]:
uint32_t input_index = 0;

### 3. Evaluate input, output, p2sh and witness script runs.

Run following cell to evaluate the `previous output` | `input` script(s).

In [8]:
bc::code ec = evaluate_in_out_p2sh_witness_runs(previous_output_script, previous_output_amount,
  spending_transaction, input_index, bc::machine::rule_fork::all_rules);
std::cout << ec.message() << std::endl;


=========== Input script evaluation ===========

> Operation: 0
[00149a19a31c2fda7d0c30215ec954a20a542aa84ad3]

> Stack after operation: 0
[00149a19a31c2fda7d0c30215ec954a20a542aa84ad3]

=========== Output script evaluation ==========

> Operation: 0
hash160

> Stack after operation: 0
[dcdc2f89b96c420751e3750da7d5073a81b16946]

>> Operation: 1
[dcdc2f89b96c420751e3750da7d5073a81b16946]

>> Stack after operation: 1
[dcdc2f89b96c420751e3750da7d5073a81b16946]
[dcdc2f89b96c420751e3750da7d5073a81b16946]

>>> Operation: 2
equal

>>> Stack after operation: 2
[01]

----------- P2SH pattern detected -------------

======= P2SH Embedded script evaluation =======

> Operation: 0
zero

> Stack after operation: 0
[]

>> Operation: 1
[9a19a31c2fda7d0c30215ec954a20a542aa84ad3]

>> Stack after operation: 1
[9a19a31c2fda7d0c30215ec954a20a542aa84ad3]
[]

---------- Witness program detected -----------

============== Witness evaluation =============

> Operation: 0
dup

> Stack after operation: 0
[026c